In [1]:
import numpy as np
import pandas as pd
import os
import xarray as xr
import seaborn as sns

from n0_config_params import *
from n0bis_config_analysis_functions import *

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Load Data

In [2]:
sujet_list = np.array(['PD01','MJ02','VN03','GB04','LV05','EF06','DM08','TA09','BH10','FA11','BD12','FP13',
'MD14','LG15','GM16','JR17','SE18','TM19','TY20','ZV21','DI22','LF23','TJ24','DF25','MN26','BD27','NT28','SC29',
'AR30','HJ31','CM32','MA33']) #exclude PB07
session_list = np.array(['+', 'o', '-'])
cond_list = np.array(['FR_CV1', 'MECA', 'CO2', 'FR_CV2'])

correspondance_odor = {'+' : 'P', 'o' : '0', '-' : 'N'}
correspondance_cond = {'FR_CV1' : 'Variables cycle BL', 'MECA' : 'Variables cycle T1', 'CO2' : 'Variables cycle T2', 'FR_CV2' : 'VS POST'}

df_columns_keep = ['TI', 'Te', 'Ttot', 'BF', 'VT', 'Ve', 'VT/Ti', 'Ti/Ttot', 'PRESS', 'PetCO2', 'VAS S', 'VAS A', 'Temps de début de cycle']

In [4]:
# sujet = 'PD01'
# session_i = 'o'
# cond = 'FR_CV1'

os.chdir(os.path.join(path_data, 'respi_detection'))

for sujet in sujet_list:

    for cond in cond_list:

        for session_i in session_list:

            file_df = [file_i for file_i in os.listdir() if (file_i.find(f'{sujet}') != -1 and file_i.find(f'{correspondance_odor[session_i]}.xlsx') != -1)][0]
            raw_df = pd.read_excel(file_df, sheet_name=correspondance_cond[cond])
            raw_df = raw_df.drop(columns=['INSPI', 'EXPI'])
            raw_df = raw_df.drop([0])
            raw_df_val = raw_df['Temps de début de cycle'].values.astype('float')
            mask = ~np.isnan(raw_df_val)
            raw_df = raw_df.loc[mask,:]
            raw_df = raw_df[df_columns_keep]
            raw_df.insert(0, 'sujet', [sujet]*raw_df.index.shape[0], True)
            raw_df.insert(1, 'cond', [cond]*raw_df.index.shape[0], True)
            raw_df.insert(2, 'session', [session_i]*raw_df.index.shape[0], True)

            if (sujet, cond, session_i) == (sujet_list[0], cond_list[0], session_list[0]): 

                df_respi = raw_df

            else:

                df_respi = pd.concat([df_respi, raw_df])

df_respi = df_respi.reset_index().drop(columns=['index'])
df_respi

,sujet,cond,session,TI,Te,Ttot,BF,VT,Ve,VT/Ti,Ti/Ttot,PRESS,PetCO2,VAS S,VAS A,Temps de début de cycle
0,PD01,FR_CV1,+,3.36,3.8,7.16,8.379888,1.240674,10.396710,0.369248,46.927374,-2.762483,31.306473,NaN,NaN,0
1,PD01,FR_CV1,+,2.47,2.69,5.16,11.627907,0.954505,11.098891,0.386439,47.868217,-2.781207,35.360859,NaN,NaN,7.16
2,PD01,FR_CV1,+,2.35,2.61,4.96,12.096774,0.823481,9.961464,0.350417,47.379032,-2.650142,42.060657,NaN,NaN,12.32
3,PD01,FR_CV1,+,2.21,2.45,4.66,12.875536,0.786929,10.132132,0.356076,47.424893,-2.762483,39.932823,NaN,NaN,17.28
4,PD01,FR_CV1,+,2.08,3.44,5.52,10.869565,0.765906,8.325066,0.368224,37.681159,-2.762483,38.207554,NaN,NaN,21.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34921,MA33,FR_CV2,-,2.91,2.57,5.48,10.948905,0.511658,5.602099,0.175828,53.10219,-2.056912,38.053444,NaN,NaN,375.49
34922,MA33,FR_CV2,-,1.71,1.8,3.51,17.094017,0.239075,4.086753,0.139810,48.717949,-1.687147,23.066575,NaN,NaN,380.97
34923,MA33,FR_CV2,-,1.65,1.8,3.45,17.391304,0.286438,4.981529,0.173599,47.826087,-1.88176,39.383446,NaN,NaN,384.48
34924,MA33,FR_CV2,-,1.57,1.71,3.28,18.292683,0.297653,5.444880,0.189588,47.865854,-1.920683,22.028522,NaN,NaN,387.93


# Analyze time

In [8]:
for sujet in sujet_list:

    for cond in cond_list:

        for session_i in session_list:

            print(df_respi.query(f"sujet == '{sujet}' & cond == '{cond}' & session == '{session_i}'")['Ttot'])
            break

0     7.16
1     5.16
2     4.96
3     4.66
4     5.52
      ... 
83       0
84       0
85       0
86       0
87       0
Name: Ttot, Length: 88, dtype: object
216    3.23
217     6.5
218    5.15
219    4.27
220    4.36
       ... 
299     NaN
300     NaN
301     NaN
302     NaN
303     NaN
Name: Ttot, Length: 88, dtype: object
443    5.78
444    5.41
445    5.07
446    5.02
447    7.22
448    6.27
449    5.13
450    6.25
451    4.84
452    6.25
453    5.28
454    5.45
455     5.3
456    4.45
457    4.58
458     5.1
459    4.77
460    5.71
461    4.87
462    4.82
463    6.75
464    4.61
465    4.54
466    4.21
467    4.43
468     4.6
469    5.05
470    4.95
471     4.9
472    5.39
473    3.99
474    4.38
475     4.4
476    5.65
477    5.23
478    5.79
479    6.85
480    6.68
481    6.23
482    5.23
483    5.26
484    4.75
485    4.15
486    4.39
487    4.48
488    3.96
489    6.45
490     5.4
491    5.32
492    6.02
493    5.35
494    5.47
495    5.36
496    5.77
497     4.7
498    5.01